In [8]:
import math
import sys,os
import numpy as np
import pandas as pd

from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

In [9]:
zillow = pd.read_csv("https://raw.github.com/Ziqi-Li/FastGWR/master/Zillow-test-dataset/zillow_1k.csv",sep=',')
zillow.head()

,utmX,utmY,value,nbaths,nbeds,area,age
0,374161.333830,3.757568e+06,56.976,1.0,2.0,954.0,71.0
1,412657.822910,3.761734e+06,157.982,2.0,4.0,1614.0,45.0
2,391600.475640,3.782212e+06,512.612,4.0,4.0,2962.0,54.0
3,375056.064681,3.748579e+06,111.437,2.0,3.0,1292.0,65.0
4,388693.693038,3.779865e+06,128.327,1.0,3.0,1169.0,78.0


In [10]:
#Converting things into matrices
y = zillow.value.values.reshape(-1,1)
X = zillow.iloc[:,3:].values
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = np.array(list(zip(u,v)))

In [11]:
X = (X - np.mean(X,axis=0))/np.std(X, axis=0)
y = (y - np.mean(y,axis=0))/np.std(y, axis=0)

### `mgwr`

In [12]:
%%time
#Bandwidth searching
selector = Sel_BW(coords,y,X,multi=True)
bws = selector.search(verbose=True)
print("Optimal Bandwidths:",bws)

#Fitting the model with optimal bandwidth
pysal_result_mgwr=MGWR(coords,y,X,selector=selector).fit()

Current iteration: 1 ,SOC: 0.0054557
Bandwidths: 133.0, 52.0, 116.0, 82.0, 50.0
Current iteration: 2 ,SOC: 0.0044876
Bandwidths: 158.0, 105.0, 120.0, 55.0, 50.0
Current iteration: 3 ,SOC: 0.0026065
Bandwidths: 182.0, 127.0, 122.0, 55.0, 43.0
Current iteration: 4 ,SOC: 0.0022962
Bandwidths: 202.0, 195.0, 122.0, 55.0, 43.0
Current iteration: 5 ,SOC: 0.0016683
Bandwidths: 202.0, 285.0, 125.0, 55.0, 43.0
Current iteration: 6 ,SOC: 0.0010319
Bandwidths: 202.0, 365.0, 125.0, 55.0, 43.0
Current iteration: 7 ,SOC: 0.0013286
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 8 ,SOC: 0.0004091
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 9 ,SOC: 0.0001911
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 10 ,SOC: 0.0001056
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 11 ,SOC: 6.57e-05
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 12 ,SOC: 4.4e-05
Bandwidths: 202.0, 999.0, 125.0, 55.0, 43.0
Current iteration: 13 ,SOC: 3


CPU times: user 7min 33s, sys: 8.63 s, total: 7min 41s
Wall time: 1min 57s


In [13]:
!fastgwr --version

fastgwr, version 0.2.7


### FastGWR

In [14]:
#Running FastGWR 4 Processor
!fastgwr run -np 4 -data https://raw.github.com/Ziqi-Li/FastGWR/master/Zillow-test-dataset/zillow_2k.csv -out results_mgwr.csv -adaptive -constant -mgwr


------------------------------------------------------------
Starting FastGWR with 4 Processors
Spatial Kernel: Adaptive Bisquare
Data Input Path: https://raw.github.com/Ziqi-Li/FastGWR/master/Zillow-test-dataset/zillow_2k.csv
Output Result Path: results_mgwr.csv
Constant: True
MGWR Backfitting...
Data are standardized
Initialization Done...
Iter: 1 SOC: 4.78e-03
bws: [94.0, 44.0, 1492.0, 50.0, 44.0]
Iter: 2 SOC: 1.98e-03
bws: [167.0, 48.0, 1998.0, 56.0, 44.0]
Iter: 3 SOC: 1.26e-03
bws: [248.0, 52.0, 1998.0, 56.0, 44.0]
Iter: 4 SOC: 9.19e-04
bws: [270.0, 58.0, 1998.0, 56.0, 44.0]
Iter: 5 SOC: 7.67e-04
bws: [272.0, 66.0, 1998.0, 56.0, 44.0]
Iter: 6 SOC: 7.65e-04
bws: [280.0, 78.0, 1998.0, 56.0, 44.0]
Iter: 7 SOC: 4.17e-04
bws: [280.0, 78.0, 1998.0, 56.0, 44.0]
Iter: 8 SOC: 6.79e-04
bws: [280.0, 97.0, 1998.0, 56.0, 44.0]
Iter: 9 SOC: 6.27e-04
bws: [280.0, 113.0, 1998.0, 56.0, 44.0]
Iter: 10 SOC: 3.97e-04
bws: [280.0, 117.0, 1998.0, 56.0, 44.0]
Iter: 11 SOC: 3.30e-04
bws: [280.0, 123.0, 1

### Results validation

In [15]:
print("MGWR pysal AICc - ", pysal_result_mgwr.aicc)
print("MGWR pysal R2 - ",pysal_result_mgwr.R2)
print("MGWR pysal ENP_j - ",pysal_result_mgwr.ENP_j)

MGWR pysal AICc -  1679.0150747382786
MGWR pysal R2 -  0.7660681409024748
MGWR pysal ENP_j -  [10.13917587  1.15749374 16.97300011 42.38091109 56.32981658]


In [16]:
fastGWR_result_mgwr = pd.read_csv("results_mgwr.csv")
fastGWR_result_mgwr.head()

,index,residual,b_intercept,b_nbaths,b_nbeds,b_area,b_age,se_intercept,se_nbaths,se_nbeds,se_area,se_age
0,0.0,-0.336452,-0.046794,0.084333,-0.040595,0.629198,-0.081248,0.034153,0.053541,0.014126,0.083024,0.082652
1,1.0,-0.042770,-0.077343,0.154635,-0.040205,0.053850,0.128839,0.035849,0.079196,0.014202,0.111559,0.185932
2,2.0,-0.370762,-0.035594,0.078720,-0.040924,0.310587,-0.006906,0.033693,0.068944,0.014231,0.131940,0.108631
3,3.0,0.131690,-0.071150,0.036836,-0.039820,0.660700,0.155930,0.033063,0.063875,0.013821,0.069140,0.100749
4,4.0,-0.174278,-0.065584,0.113368,-0.040148,0.343204,-0.045846,0.033543,0.057233,0.013854,0.092935,0.071965


In [17]:
#Validate model residuals
print(np.allclose(fastGWR_result_mgwr.residual,
                  pysal_result_mgwr.resid_response.reshape(-1)))


ValueError: operands could not be broadcast together with shapes (2000,) (1000,) 

In [ ]:
#Validate parameter estimates
print(np.allclose(np.array(fastGWR_result_mgwr.iloc[:,2:7]),
                  pysal_result_mgwr.params))



In [ ]:
#Validate parameter estimates standard errors
print(np.allclose(np.array(fastGWR_result_mgwr.iloc[:,7:12]),
                  pysal_result_mgwr.bse,atol=1e-3))
